# EDA

In [1]:
# Import python packages
from pathlib import Path
import zipfile
import kaggle
import pandas as pd
kaggle.api.authenticate()

## 1. Data acquisition

### 1.2. Download

In [2]:
# Create target directory
Path('../data/raw').mkdir(parents=True, exist_ok=True)

# Download latest version
kaggle.api.dataset_download_files(
    'bhanupratapbiswas/superstore-sales',
    path='../data/raw/'
)

Dataset URL: https://www.kaggle.com/datasets/bhanupratapbiswas/superstore-sales


### 1.2. Extract

In [3]:
with zipfile.ZipFile('../data/raw/superstore-sales.zip', mode='r') as archive:
    archive.extract('superstore_final_dataset (1).csv', '../data/raw')

### 1.3. Inspect

In [4]:
data_df = pd.read_csv('../data/raw/superstore_final_dataset (1).csv', encoding='unicode_escape')
data_df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold N Roll Cart System,22.3680


In [5]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row_ID         9800 non-null   int64  
 1   Order_ID       9800 non-null   object 
 2   Order_Date     9800 non-null   object 
 3   Ship_Date      9800 non-null   object 
 4   Ship_Mode      9800 non-null   object 
 5   Customer_ID    9800 non-null   object 
 6   Customer_Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal_Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product_ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub_Category   9800 non-null   object 
 16  Product_Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

## 2. Feature selection

In [6]:
data_df.drop(columns=['Row_ID'], inplace=True)

## 3. Factor levels

In [7]:
categorical_features = ['Order_ID', 'Order_Date', 'Ship_Date','Ship_Mode','Customer_ID',
                        'Customer_Name', 'Segment', 'Country', 'City', 'State',
                        'Postal_Code', 'Region', 'Product_ID', 'Category', 'Sub_Category',
                        'Product_Name']

data_df[categorical_features].nunique()

Order_ID         4922
Order_Date       1230
Ship_Date        1326
Ship_Mode           4
Customer_ID       793
Customer_Name     793
Segment             3
Country             1
City              529
State              49
Postal_Code       626
Region              4
Product_ID       1861
Category            3
Sub_Category       17
Product_Name     1849
dtype: int64

In [8]:
# Drop customer name - it's redundant to customer ID
data_df.drop(columns='Customer_Name', inplace=True)
categorical_features.remove('Customer_Name')
data_df[categorical_features].nunique()


Order_ID        4922
Order_Date      1230
Ship_Date       1326
Ship_Mode          4
Customer_ID      793
Segment            3
Country            1
City             529
State             49
Postal_Code      626
Region             4
Product_ID      1861
Category           3
Sub_Category      17
Product_Name    1849
dtype: int64

In [9]:
# Drop country - it's constant
data_df.drop(columns='Country', inplace=True)
categorical_features.remove('Country')
data_df[categorical_features].nunique()

Order_ID        4922
Order_Date      1230
Ship_Date       1326
Ship_Mode          4
Customer_ID      793
Segment            3
City             529
State             49
Postal_Code      626
Region             4
Product_ID      1861
Category           3
Sub_Category      17
Product_Name    1849
dtype: int64

## 3. Encoding

**One-hot encoding** (low cardinality)
1. `Ship_Mode`
2. `Segment`
3. `Region`
4. `Category`

**Ordinal encoding** (high cardinality)
1. `Order_ID`
2. `Order_Date`
3. `Ship_Date`
4. `Customer_ID`
5. `City`
6. `State`
7. `Postal_Code`
8. `Product_ID`
9. `Sub_Category`
10. `Product_Name`